#1.根据statistics_valid_data.txt，针对每一个要素删选出排名前300的站点
#2.每一个要素一个文件，生成完整的3-hourly时间序列。


In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import datetime
import pandas as pd
txt_path='/home/liuli/GTS_GFS_MOS/domestic_hourly_data/statistics_valid_data_rain_0.8.txt'
file1= pd.read_csv(txt_path,sep='\t',header=None,names=['sta','para','rate'])
df1=pd.DataFrame(file1)
print(df1.info())
df1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2465 entries, 0 to 2464
Data columns (total 3 columns):
sta     2465 non-null int64
para    2465 non-null object
rate    2465 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 57.9+ KB
None


,sta,para,rate
0,99217,pr6,0.856263
1,57403,pr6,0.918549
2,70414,pr6,0.863792
3,91212,pr6,0.857632
4,91408,pr6,0.822040
5,47003,pr6,0.863107
6,47008,pr6,0.868583
7,47014,pr6,0.898015
8,47016,pr6,0.850103
9,47020,pr6,0.852156


In [2]:
data_path='/home/liuli/GTS_GFS_MOS/domestic_hourly_data/df_final_1100(rain).csv'
file0= pd.read_csv(data_path,sep=',')
df0=pd.DataFrame(file0)
df0=df0.replace(999999.0, np.nan) 
df0=df0.replace(999998.0, np.nan)
df0=df0.replace(9999.0, np.nan)
df0['sta']=df0['sta'].astype('str')
#df0['sta']=df0['sta'].replace('NaN','53603')
df0['time']=pd.DatetimeIndex(df0['time'])

start='2018-01-01 00:00:00'
end_of_db = '2019-01-01 00:00:00'
date_index = pd.date_range(start, end_of_db, freq = '6h')
date_df = pd.DataFrame(date_index,columns=['time'])
df0 = pd.merge(date_df,df0, on=['time'], how='left') 


print(df0.info())
print(df0.count())


/home/liuli/anaconda3/envs/MOS/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 7547526 entries, 0 to 7547525
Data columns (total 14 columns):
time    datetime64[ns]
sta     object
ht      float64
lat     float64
lon     float64
pr1     float64
pr24    float64
pr6     float64
ps      float64
psl     float64
t2m     float64
td      float64
wdir    float64
wspd    float64
dtypes: datetime64[ns](1), float64(12), object(1)
memory usage: 863.7+ MB
None
time    7547526
sta     7547526
ht      5774025
lat     5926247
lon     5926247
pr1     3993983
pr24    4318105
pr6     4036309
ps      5421110
psl     5413749
t2m     5829005
td      5450439
wdir    5795054
wspd    5826611
dtype: int64


In [4]:
#每一个要素中，按照rate排名前300来筛选站点
#附属产品：所有6个要素的站点信息，放在一个txt文档里面，为以后GFS提取数据使用
name='rain-station-info.txt'
f=open(name,'w')
cols=['pr6']
print(cols)
#print(df0[df0['sta'].isin(['54578'])].dropna(subset=['lat','lon'],how='any'))
for i in cols:
    df_final=pd.DataFrame(columns=['time','sta','lat','lon',i])
    #print(df_final)
    df2=df1[df1['para'].isin([i])] #先挑出这个要素
    #print('df2=',df2)
    df2=df2.sort_values(by='rate',ascending=False)#把这个站点按照rate顺序排列  
    df2=df2.reset_index(drop=True)
    #print(df2.sta[:300])
   
    for j in df2.sta[:300]:   #只挑前300个站点
        print(j)
        df3=df0[df0['sta'].isin([str(j)])]
        df3=df3.reset_index(drop=True)
        print('df3=',df3)
        df4=df3[['time','sta','lat','lon',i]]
        df5=df4.dropna(subset=['lat','lon'],how='any')
        df5=df5.reset_index(drop=True)
        #print(df5)
        #print(str(df5.loc[0,'lat']))
        line=[]
        line=str(j)+'\t'+str(df5.loc[0,'lat'])+'\t'+str(df5.loc[0,'lon'])+'\n' #站号、经纬度
        f.write(line)
        df_final=pd.concat([df_final,df4],ignore_index=True)            
        del df3
        del df4
        del df5
        del line
    df_final.to_csv('./df_'+i+'.csv', sep=',', header=True, index=False)    
    del df_final
    del df2

['pr6']
56684
df3=                     time    sta      ht   lat     lon       pr1      pr24  \
0    2018-01-01 00:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
1    2018-01-01 06:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
2    2018-01-01 12:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
3    2018-01-01 18:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
4    2018-01-02 00:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
5    2018-01-02 06:00:00  56684  2188.3  26.4  103.25       0.0       0.0   
6    2018-01-02 12:00:00  56684  2188.3  26.4  103.25  999990.0  999990.0   
7    2018-01-02 18:00:00  56684  2188.3  26.4  103.25       0.1       0.1   
8    2018-01-03 00:00:00  56684  2188.3  26.4  103.25       0.2       2.0   
9    2018-01-03 06:00:00  56684  2188.3  26.4  103.25       0.3       5.7   
10   2018-01-03 12:00:00  56684  2188.3  26.4  103.25       0.8      13.7   
11   2018-01-03 18:00:00  56684  2188.3  26.4  103.25    

df3=                     time    sta    ht    lat     lon  pr1      pr24       pr6  \
0    2018-01-01 00:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
1    2018-01-01 06:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
2    2018-01-01 12:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
3    2018-01-01 18:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
4    2018-01-02 00:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
5    2018-01-02 06:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
6    2018-01-02 12:00:00  57271  87.7  32.52  112.38  0.0  999990.0  999990.0   
7    2018-01-02 18:00:00  57271  87.7  32.52  112.38  0.0       0.0       0.0   
8    2018-01-03 00:00:00  57271  87.7  32.52  112.38  0.0       0.3       0.0   
9    2018-01-03 06:00:00  57271  87.7  32.52  112.38  0.0       4.5       4.2   
10   2018-01-03 12:00:00  57271  87.7  32.52  112.38  0.0       9.4       4.9   
11   2018-01-03 18:00:0

df3=                     time    sta     ht    lat     lon  pr1      pr24  \
0    2018-01-01 00:00:00  57303  459.9  31.73  105.92  0.0       0.0   
1    2018-01-01 06:00:00  57303  459.9  31.73  105.92  0.0       0.0   
2    2018-01-01 12:00:00  57303  459.9  31.73  105.92  0.0       0.0   
3    2018-01-01 18:00:00  57303  459.9  31.73  105.92  0.0       0.0   
4    2018-01-02 00:00:00  57303  459.9  31.73  105.92  0.0       0.0   
5    2018-01-02 06:00:00  57303  459.9  31.73  105.92  0.0       0.0   
6    2018-01-02 12:00:00  57303  459.9  31.73  105.92  0.0       0.0   
7    2018-01-02 18:00:00  57303  459.9  31.73  105.92  0.0       0.0   
8    2018-01-03 00:00:00  57303  459.9  31.73  105.92  0.0       0.3   
9    2018-01-03 06:00:00  57303  459.9  31.73  105.92  0.2       1.7   
10   2018-01-03 12:00:00  57303  459.9  31.73  105.92  0.5       3.4   
11   2018-01-03 18:00:00  57303  459.9  31.73  105.92  0.9       9.6   
12   2018-01-04 00:00:00  57303  459.9  31.73  105.92  0.0 

df3=                     time    sta     ht    lat     lon  pr1      pr24  \
0    2018-01-01 00:00:00  57297  114.5  32.13  114.05  0.0       0.0   
1    2018-01-01 06:00:00  57297  114.5  32.13  114.05  0.0       0.0   
2    2018-01-01 12:00:00  57297  114.5  32.13  114.05  0.0       0.0   
3    2018-01-01 18:00:00  57297  114.5  32.13  114.05  0.0       0.0   
4    2018-01-02 00:00:00  57297  114.5  32.13  114.05  0.0       0.0   
5    2018-01-02 06:00:00  57297  114.5  32.13  114.05  0.0       0.0   
6    2018-01-02 12:00:00  57297  114.5  32.13  114.05  0.0       0.0   
7    2018-01-02 18:00:00  57297  114.5  32.13  114.05  0.1       0.3   
8    2018-01-03 00:00:00  57297  114.5  32.13  114.05  1.0       3.6   
9    2018-01-03 06:00:00  57297  114.5  32.13  114.05  0.9       9.2   
10   2018-01-03 12:00:00  57297  114.5  32.13  114.05  1.6      21.1   
11   2018-01-03 18:00:00  57297  114.5  32.13  114.05  1.7      34.4   
12   2018-01-04 00:00:00  57297  114.5  32.13  114.05  1.6 

df3=                     time    sta    ht    lat     lon  pr1      pr24       pr6  \
0    2018-01-01 00:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
1    2018-01-01 06:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
2    2018-01-01 12:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
3    2018-01-01 18:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
4    2018-01-02 00:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
5    2018-01-02 06:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
6    2018-01-02 12:00:00  57296  34.2  32.35  114.67  0.0       0.0       0.0   
7    2018-01-02 18:00:00  57296  34.2  32.35  114.67  0.2       0.4       0.4   
8    2018-01-03 00:00:00  57296  34.2  32.35  114.67  0.7       2.6       2.2   
9    2018-01-03 06:00:00  57296  34.2  32.35  114.67  0.9       7.5       4.9   
10   2018-01-03 12:00:00  57296  34.2  32.35  114.67  1.7      17.5      10.0   
11   2018-01-03 18:00:0

...

df3=                     time    sta      ht    lat     lon  pr1  pr24  pr6     ps  \
0    2018-01-01 00:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  750.5   
1    2018-01-01 06:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  748.3   
2    2018-01-01 12:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  748.7   
3    2018-01-01 18:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  749.8   
4    2018-01-02 00:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  750.1   
5    2018-01-02 06:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  749.3   
6    2018-01-02 12:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  749.0   
7    2018-01-02 18:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  749.3   
8    2018-01-03 00:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  747.6   
9    2018-01-03 06:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  746.3   
10   2018-01-03 12:00:00  56565  2516.8  27.44  101.49  0.0   0.0  0.0  745.3   
11   2018-01-03 18:00:0

df3=                     time    sta      ht    lat    lon  pr1      pr24  \
0    2018-01-01 00:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
1    2018-01-01 06:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
2    2018-01-01 12:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
3    2018-01-01 18:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
4    2018-01-02 00:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
5    2018-01-02 06:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
6    2018-01-02 12:00:00  56533  1583.3  27.75  98.67  0.0       0.0   
7    2018-01-02 18:00:00  56533  1583.3  27.75  98.67  1.5       3.0   
8    2018-01-03 00:00:00  56533  1583.3  27.75  98.67  1.1       7.6   
9    2018-01-03 06:00:00  56533  1583.3  27.75  98.67  0.3      13.0   
10   2018-01-03 12:00:00  56533  1583.3  27.75  98.67  0.1      15.3   
11   2018-01-03 18:00:00  56533  1583.3  27.75  98.67  0.0      12.4   
12   2018-01-04 00:00:00  56533  1583.3  27.75  98.67  0.1 

df3=                     time    sta     ht    lat    lon  pr1  pr24  pr6     ps  \
0    2018-01-01 00:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  956.6   
1    2018-01-01 06:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  953.2   
2    2018-01-01 12:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  953.6   
3    2018-01-01 18:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  954.4   
4    2018-01-02 00:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  953.9   
5    2018-01-02 06:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  953.0   
6    2018-01-02 12:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  955.2   
7    2018-01-02 18:00:00  56498  549.0  28.15  104.5  0.0   0.0  0.0  956.9   
8    2018-01-03 00:00:00  56498  549.0  28.15  104.5  0.1   0.4  0.4  957.3   
9    2018-01-03 06:00:00  56498  549.0  28.15  104.5  0.1   2.1  1.7  955.9   
10   2018-01-03 12:00:00  56498  549.0  28.15  104.5  0.3   3.9  1.8  955.9   
11   2018-01-03 18:00:00  56498  549.0  28.15  

df3=                     time    sta     ht   lat     lon  pr1  pr24  pr6     ps  \
0    2018-01-01 00:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  973.5   
1    2018-01-01 06:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  970.3   
2    2018-01-01 12:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  970.4   
3    2018-01-01 18:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  971.3   
4    2018-01-02 00:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  970.8   
5    2018-01-02 06:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  970.2   
6    2018-01-02 12:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  972.4   
7    2018-01-02 18:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  973.5   
8    2018-01-03 00:00:00  56496  399.4  28.3  105.23  0.0   0.0  0.0  973.8   
9    2018-01-03 06:00:00  56496  399.4  28.3  105.23  0.1   0.4  0.4  972.9   
10   2018-01-03 12:00:00  56496  399.4  28.3  105.23  0.9   4.4  4.0  973.3   
11   2018-01-03 18:00:00  56496  399.4  28.3  1

df3=                     time    sta     ht    lat     lon  pr1      pr24  \
0    2018-01-01 00:00:00  56493  292.8  28.85  104.97  0.0       0.0   
1    2018-01-01 06:00:00  56493  292.8  28.85  104.97  0.0       0.0   
2    2018-01-01 12:00:00  56493  292.8  28.85  104.97  0.0       0.0   
3    2018-01-01 18:00:00  56493  292.8  28.85  104.97  0.0       0.0   
4    2018-01-02 00:00:00  56493  292.8  28.85  104.97  0.0       0.0   
5    2018-01-02 06:00:00  56493  292.8  28.85  104.97  0.0       0.0   
6    2018-01-02 12:00:00  56493  292.8  28.85  104.97  0.0       0.0   
7    2018-01-02 18:00:00  56493  292.8  28.85  104.97  0.2       0.2   
8    2018-01-03 00:00:00  56493  292.8  28.85  104.97  0.3       3.8   
9    2018-01-03 06:00:00  56493  292.8  28.85  104.97  0.1       4.8   
10   2018-01-03 12:00:00  56493  292.8  28.85  104.97  0.1       6.0   
11   2018-01-03 18:00:00  56493  292.8  28.85  104.97  0.0       6.1   
12   2018-01-04 00:00:00  56493  292.8  28.85  104.97  0.0 